In [9]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

from services import content_preparator as chunker

ImportError: cannot import name 'content_preparator' from 'services' (/home/godwin/Desktop/Organised/python/python/ML_project_regression/.venv/lib/python3.9/site-packages/services/__init__.py)

In [7]:
loader = TextLoader("lettre_1.txt")
pages = loader.load_and_split()

unify_content = ""
for page in pages:
    unify_content += "\n" + page.page_content

print(unify_content)


Profession : sans
Taille: 158
G : 3 P : 2 
Sérologie :
/RAI:	Rub +; Toxo -(09/16); CMV: +; 
Ag HBs nég/Ac nég/ anti-core nég (04/16);  HbC-(04/16); Syph -(04/16); HIV -(04/16); Chlamydia -;
Agglu irr. - (08/16);  

Ménarche: 12
Règles: +- réguliers  
Antécédents obstétricaux : 21/03/07: 39w6C/S pour SFA F2810g Alyena AM
2010: accht eutocique 
Antécédents médicaux : 
gastrite
Risques : 1 Césarienne dans antécédents
Fibrinogène: 2.86 ( 12/08/16) 

Grossesses antérieures : 

Grossesse n° 1 	suivie par: Dr. Alain Claudot 	accouchée par: Dr. A. Loos  
Terme prévu: 22/03/2007 	Fin de grossesse le 21/03/2007 à 39 s. 6 j. 
Résumé de la grossesse:
rhogam 10/11 sur métro, refait le 5/1, Anémie sidéroprive, traitée par supplément martial  
Accouchement: Présentation: céphalique 
Motif d'entrée: Rupture prématurée des membranes avec entrée en travail spontané dans les 24 heures  
Travail: optimalisé en salle de travail par perfusion de syntocinon® rupture spontanée des membranes laissant s'écoule

In [5]:
# recursive_splitter = chunker.recursive_splitter(chunk_size=512,overlap=20)

## For this method we need to define an embedder
embedder = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
semantic_splitter = chunker.semantic_chunker(embedder)
semantic_splitter_openai = chunker.semantic_chunker(OpenAIEmbeddings())

In [ ]:
semantic_splited_chunks = semantic_splitter.create_documents([unify_content])
